向量化存储github仓库代码
进行QA问答

In [1]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain_community.embeddings.cloudflare_workersai import CloudflareWorkersAIEmbeddings
from supabase.client import Client, create_client

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0.7,
    streaming=True,
)
embeddings = CloudflareWorkersAIEmbeddings(
    account_id=os.getenv('CF_ACCOUNT_ID'),
    api_token=os.getenv('CF_API_TOKEN'),
    model_name="@cf/baai/bge-large-en-v1.5",
)

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")

supabase: Client = create_client(supabase_url, supabase_key)

In [33]:
from langchain_community.llms.cloudflare_workersai import CloudflareWorkersAI

account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')
print(account_id)
print(api_token)

# CloudflareWorkersAI
model = '@cf/meta/llama-3-8b-instruct'
cf_llm = CloudflareWorkersAI(
    account_id=account_id,
    api_token=api_token,
    model=model
)

8483c3ec7a0cbc54a8d660b5b9002b04
Gcllof8ze6dgtcqFI5FQZ2SD_5tfCD4Db7NuS6jn


In [2]:
import os
from langchain.document_loaders import TextLoader

root_dir = '/Users/pangmengting/Documents/workspace/chat-ollama_rag'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try:
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e:
            pass

In [3]:
len(docs)

502

In [ ]:
# docs

[Document(page_content='<template>\n  <div class="h-full">\n    <NuxtLayout>\n      <NuxtPage />\n    </NuxtLayout>\n    <USlideovers />\n    <UModals />\n  </div>\n</template>', metadata={'source': '/Users/pangmengting/Documents/workspace/chat-ollama_rag/app.vue'}),
 Document(page_content="lockfileVersion: '9.0'\n\nsettings:\n  autoInstallPeers: true\n  excludeLinksFromLockfile: false\n\nimporters:", metadata={'source': '/Users/pangmengting/Documents/workspace/chat-ollama_rag/pnpm-lock.yaml'}),
 Document(page_content=".:\n    dependencies:\n      '@google/generative-ai':\n        specifier: ^0.11.1\n        version: 0.11.1\n      '@iconify-json/heroicons':\n        specifier: ^1.1.21\n        version: 1.1.21\n      '@iconify-json/iconoir':\n        specifier: ^1.1.44\n        version: 1.1.44\n      '@iconify-json/material-symbols':\n        specifier: ^1.1.81\n        version: 1.1.81\n      '@iconify-json/mdi':\n        specifier: ^1.1.66\n        version: 1.1.66\n      '@iconify-json

In [4]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, add_start_index=True)
splits = text_splitter.split_documents(docs)

2024-07-04 17:25:19,327:WARNING - Created a chunk of size 1269, which is longer than the specified 1000
2024-07-04 17:25:19,328:WARNING - Created a chunk of size 1569, which is longer than the specified 1000
2024-07-04 17:25:19,329:WARNING - Created a chunk of size 1653, which is longer than the specified 1000
2024-07-04 17:25:19,330:WARNING - Created a chunk of size 2845, which is longer than the specified 1000
2024-07-04 17:25:19,331:WARNING - Created a chunk of size 2348, which is longer than the specified 1000
2024-07-04 17:25:19,331:WARNING - Created a chunk of size 2990, which is longer than the specified 1000
2024-07-04 17:25:19,333:WARNING - Created a chunk of size 2660, which is longer than the specified 1000
2024-07-04 17:25:19,334:WARNING - Created a chunk of size 1015, which is longer than the specified 1000
2024-07-04 17:25:19,334:WARNING - Created a chunk of size 2848, which is longer than the specified 1000
2024-07-04 17:25:19,335:WARNING - Created a chunk of size 1062, 

In [5]:
len(splits)

1113

In [8]:
from langchain_community.vectorstores import SupabaseVectorStore

vectorstore = SupabaseVectorStore.from_documents(
    splits,
    embeddings,
    client=supabase,
    table_name="bge_large_vector",
    query_name="bge_large_match_documents",
)

2024-07-04 17:34:12,906:INFO - HTTP Request: POST https://infrxrfaftyrxvkwvncf.supabase.co/rest/v1/bge_large_vector?columns=%22metadata%22%2C%22id%22%2C%22content%22%2C%22embedding%22 "HTTP/1.1 201 Created"
2024-07-04 17:34:37,930:INFO - HTTP Request: POST https://infrxrfaftyrxvkwvncf.supabase.co/rest/v1/bge_large_vector?columns=%22metadata%22%2C%22id%22%2C%22content%22%2C%22embedding%22 "HTTP/1.1 201 Created"
2024-07-04 17:34:52,877:INFO - HTTP Request: POST https://infrxrfaftyrxvkwvncf.supabase.co/rest/v1/bge_large_vector?columns=%22metadata%22%2C%22id%22%2C%22content%22%2C%22embedding%22 "HTTP/1.1 201 Created"


In [ ]:
from langchain.vectorstores import Chroma

persist_directory = '/Users/pangmengting/Documents/workspace/python-learning/langchain/rag/data'
collection_name = 'chat-ollama-index'
# 创建
vectorstore = Chroma.from_documents(
    splits,
    embeddings,
    collection_name=collection_name,
    persist_directory=persist_directory
)

In [15]:
retriever = vectorstore.as_retriever()
# retriever.search_kwargs['distance_metric'] = 'cos'
# retriever.search_kwargs['fetch_k'] = 100
# retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 10

In [49]:
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain

# from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(qw_llm_openai, retriever=retriever)

In [17]:
questions = [
    "如何进行向量化并存储?",
    "在相似性查询的时候，是如何操作的?"
]
chat_history = []

for question in questions:
    result = qa.invoke({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"Question:\n {question} \n")
    print(f"Answer:\n {result['answer']} \n\n")

2024-07-04 17:37:19,954:INFO - HTTP Request: POST https://infrxrfaftyrxvkwvncf.supabase.co/rest/v1/rpc/bge_large_match_documents?limit=10 "HTTP/1.1 200 OK"
2024-07-04 17:37:20,015:INFO - Retrying request to /chat/completions in 0.926833 seconds
2024-07-04 17:37:20,948:INFO - Retrying request to /chat/completions in 1.914957 seconds


APIConnectionError: Connection error.

In [39]:
def ask(question, chat_history):
    response = qa({"question": question, "chat_history": chat_history})
    print(f"Question:\n {question}\n")
    print(f"Answer:\n {response['answer']}\n")

In [25]:
retriever.invoke("如何进行向量化并存储?")

2024-07-04 17:41:06,559:INFO - HTTP Request: POST https://infrxrfaftyrxvkwvncf.supabase.co/rest/v1/rpc/bge_large_match_documents?limit=10 "HTTP/1.1 200 OK"


[Document(page_content='如果您想在最新的代码库中运行，并且可以实时应用更改，clone 该存储库，并按照以下步骤进行：\n\n1. 安装 Ollama 服务器\n\n    您需要运行 Ollama 服务器。按照 [Ollama](https://github.com/ollama/ollama) 的安装指南进行安装。默认情况下，它运行在 http://localhost:11434。\n\n2. 安装 Chroma\n\n    请参阅 [https://docs.trychroma.com/getting-started](https://docs.trychroma.com/getting-started) 获取 Chroma 安装指南。\n\n    我们建议在 Docker 容器中运行：\n\n    ```bash\n    # https://hub.docker.com/r/chromadb/chroma/tags\n\n    docker pull chromadb/chroma\n    docker run -d -p 8000:8000 chromadb/chroma\n    ```\n    现在，ChromaDB 正在运行于 http://localhost:8000\n\n3. ChatOllama 设置\n\n    现在，我们可以完成必要的设置，以便运行 ChatOllama。\n\n    3.1 复制 `.env.example` 文件到 `.env` 文件：\n\n    ```bash\n    cp .env.example .env\n    ```\n\n    3.2 确保安装依赖项：\n\n    ```bash\n    pnpm install\n    ```\n\n    3.3 运行迁移命令以创建数据库表：\n\n    ```bash\n    pnpm prisma-migrate\n    ```\n\n4. 启动开发服务器\n\n    > 确保 __[Ollama Server](#ollama-server)__  服务器和 __[ChromaDB](#install-chromadb-and-startup)__  都正在运行。\n\n    启动开发服务器在 `

In [40]:
qa({"question": "如何进行向量化并存储?", "chat_history": chat_history})

2024-07-04 17:47:49,000:INFO - HTTP Request: POST https://infrxrfaftyrxvkwvncf.supabase.co/rest/v1/rpc/bge_large_match_documents?limit=10 "HTTP/1.1 200 OK"


ValueError: Request failed with status 400

In [31]:
ask("如何进行向量化并存储?", '')

2024-07-04 17:44:31,990:INFO - HTTP Request: POST https://infrxrfaftyrxvkwvncf.supabase.co/rest/v1/rpc/bge_large_match_documents?limit=10 "HTTP/1.1 200 OK"
2024-07-04 17:44:32,013:INFO - Retrying request to /chat/completions in 0.807730 seconds
2024-07-04 17:44:32,829:INFO - Retrying request to /chat/completions in 1.581052 seconds


APIConnectionError: Connection error.

In [51]:
ask('Show me the public functions of class Client', chat_history)

2024-07-04 17:51:47,201:INFO - HTTP Request: POST https://infrxrfaftyrxvkwvncf.supabase.co/rest/v1/rpc/bge_large_match_documents?limit=10 "HTTP/1.1 200 OK"
2024-07-04 17:51:47,223:INFO - Retrying request to /chat/completions in 0.900971 seconds
2024-07-04 17:51:48,131:INFO - Retrying request to /chat/completions in 1.680101 seconds


APIConnectionError: Connection error.

In [50]:
ask('How to vectorize and store?', chat_history)

2024-07-04 17:51:38,085:INFO - HTTP Request: POST https://infrxrfaftyrxvkwvncf.supabase.co/rest/v1/rpc/bge_large_match_documents?limit=10 "HTTP/1.1 200 OK"
2024-07-04 17:51:38,107:INFO - Retrying request to /chat/completions in 0.781601 seconds
2024-07-04 17:51:38,896:INFO - Retrying request to /chat/completions in 1.521070 seconds


APIConnectionError: Connection error.

In [55]:
api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')
print(api_key)
print(base_url)

# openai/moonshot
ms_llm = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


In [56]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    ms_llm, retriever, contextualize_q_prompt
)

In [59]:
ms_llm.invoke("自我介绍下")

2024-07-04 17:58:37,771:INFO - HTTP Request: POST https://api.moonshot.cn/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='你好！我是一个AI助手，专门为人们提供帮助和信息。我可以通过文字交流，回答各种问题，提供有关各种主题的信息，如科学、历史、文化、技术等。无论是解答简单的问题还是提供复杂的解释，我都会尽力提供最准确和有用的信息。\n\n同时，我会继续学习和改进，以更好地满足用户的需求。我的存在是为了帮助人们更轻松地获取信息，提高生活效率。如果你有任何问题，随时都可以向我提问。我会尽我所能为你提供帮助。', response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 11, 'total_tokens': 114}, 'model_name': 'moonshot-v1-8k', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-18afb66d-15ec-4b27-bdab-3b1299ad4203-0', usage_metadata={'input_tokens': 11, 'output_tokens': 103, 'total_tokens': 114})

In [57]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(qw_llm_openai, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [58]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "什么是rag?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

2024-07-04 17:58:07,374:INFO - HTTP Request: POST https://infrxrfaftyrxvkwvncf.supabase.co/rest/v1/rpc/bge_large_match_documents?limit=10 "HTTP/1.1 200 OK"
2024-07-04 17:58:07,423:INFO - Retrying request to /chat/completions in 0.901202 seconds
2024-07-04 17:58:08,328:INFO - Retrying request to /chat/completions in 1.780613 seconds


APIConnectionError: Connection error.

In [ ]:

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])